# Classificando carros

## Agenda
         1 - Exploração de Dados
         2 - Codificando features
                 zip() pequeno exemplo - basta ver o que está acontecendo
         3 - Treinar o modelo
                 subconjuntos de treinamento e teste Exemplo Curto - basta ver o que está acontecendo
             3.1 Implementando K-NN
         4 - Previsões vs valores reais

In [1]:
# Manipulação de dados
import pandas as pd
# Python numérico
import numpy as np

# Machine Learning
import sklearn
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model, preprocessing

In [2]:
data = pd.read_csv('car.data')

In [6]:
data.head()

,buying,maint,door,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


## 1 - Exploração de Dados

In [7]:
# Número de linhas e colunas

data.shape

(1728, 7)

In [8]:
# Informações básicas sobre todas as colunas (sem valores nulos, isso é bom hehe)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   door      1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [16]:
# Estatísticas básicas sobre colunas numéricas (sendo um conjunto de dados 'classificador', temos poucos valores únicos)

data.describe()

,buying,maint,door,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,5,3,3,3,4
top,med,med,4,4,big,med,unacc
freq,432,432,432,576,576,576,1210


## 2 - Codificando features             
        Queremos evitar o uso de features com dados não numéricos, porque iremos
        fazer cálculos com eles e não podemos fazer operações com dados que não 
        sejam numéricos. Isso significa que teremos que converter esses dados 
        não numéricos em dados numéricos.

**Aqui é onde o 'preprocessing' do sklearn atua. Seu método pode nos ajudar nessa tarefa** 

In [21]:
# LabelEncoder() será responsável por transformar os labels em valores inteiros apropriados

label_encoder = preprocessing.LabelEncoder()

**Codificando as features categóricas em valores numéricos**

In [25]:
# fit_transform() pega uma lista(cada_uma_das['colunas']) e nos retorna um vetor contendo nossos novos valores numéricos.

buying = label_encoder.fit_transform(list(data['buying']))
maint = label_encoder.fit_transform(list(data["maint"]))
door = label_encoder.fit_transform(list(data["door"]))
persons = label_encoder.fit_transform(list(data["persons"]))
lug_boot = label_encoder.fit_transform(list(data["lug_boot"]))
safety = label_encoder.fit_transform(list(data["safety"]))
class_ = label_encoder.fit_transform(list(data["class"]))

Resultado:

In [40]:
data.buying[[0,1,100,800,900,1400]]

0       vhigh
1       vhigh
100     vhigh
800      high
900       med
1400      low
Name: buying, dtype: object

In [38]:
# Hehe
buying[[0,1,100,800,900,1400]]

array([3, 3, 3, 0, 2, 1], dtype=int64)

**Agora, recombinaremos nossos dados em uma lista de features e uma lista de labels. Podemos usar a função zip() para facilitar as coisas**

        The zip() function returns a zip object, which is an iterator of tuples
        where the first item in each passed iterator is paired together, and then
        the second item in each passed iterator are paired together etc.

        A função zip() retorna um objeto zip, que é um iterador de tuplas em que o
        primeiro item em cada iterador é 'pareado' e, em seguida, o segundo item e 
        assim em diante. Para facilitar o entendimento veja o exemplo abaixo.

### |----- _zip() pequeno exemplo - basta ver o que está acontecendo_ -----|

In [56]:
fruits = ("Apple", "Banana", "Lemon")
fruit_colors = ("Red", "Yellow", "Green")

zip_test = zip(fruits, fruit_colors)

print(tuple(zip_test)) # Aqui, precisamos convertê-lo como uma tupla para mostrar os elementos

(('Apple', 'Red'), ('Banana', 'Yellow'), ('Lemon', 'Green'))


**|----- _Fim do exemplo [ \o/ ]_ ------|**

In [49]:
predict = 'class'

# Convertendo nossos dados numéricos em uma lista, para que possamos aplicar métodos de ML neles
features_X = list(zip(buying, maint, door, persons, lug_boot, safety))
labels_y = list(class_)

In [52]:
# Assim como no exemplo
features_X[0:4]

[(3, 3, 0, 0, 2, 1),
 (3, 3, 0, 0, 2, 2),
 (3, 3, 0, 0, 2, 0),
 (3, 3, 0, 0, 1, 1)]

## 3 - Treinar modelo

In [59]:
'''  Dividindo as features e os labels em subconjuntos aleatórios de treino e teste '''

features_X_train, features_X_test, labels_y_train, labels_y_test = sklearn.model_selection.train_test_split(features_X,
                                                                                                           labels_y,
                                                                                                           test_size=0.1)
# 0.1 (10%) dos dados estão sendo alocados como dados de teste, 
# enquanto os outros 90% estão sendo tratados como dados de treinamento

**[features_X_train e labels_y_train] serão usados para treinar nosso modelo**<br>
(e fazer a máquina aprender)

**[features_X_test e labels_y_test] serão usados para testar a precisão do nosso modelo**<br>
(proporção do número de previsões corretas para o número total de amostras de fornecidas)

### |----- _subconjuntos de treinamento e teste Exemplo Curto - basta ver o que está acontecendo_ -----|

In [2]:
''' Criando valores para X e y '''
X , y = np.arange(10).reshape((5, 2)), np.arange(5)
print('X:\n',X)
print('y:\n',y)

X:
 [[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
y:
 [0 1 2 3 4]


In [4]:
''' Isso que acontece '''

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

print('Valores de X_train values:\n',X_train)
print('\nValores de y_train values:\n',y_train)
print('\nValores de X_test values:\n',X_test)
print('\nValores de y_test values:\n',y_test)

Valores de X_train values:
 [[2 3]
 [4 5]
 [6 7]
 [8 9]]

Valores de y_train values:
 [1 2 3 4]

Valores de X_test values:
 [[0 1]]

Valores de y_test values:
 [0]


#### |----- _Fim do exemplo [ \o/ ]_ -----|

### 3.1 - Implementando K-Nearest Neighbors

In [97]:
# O objeto KNeighborsClassifier() recebe 1 parâmetro (o número de vizinhos que queremos)

classifier_model = KNeighborsClassifier(n_neighbors=5)

In [98]:
# Treinando o modelo

classifier_model.fit(features_X_train, labels_y_train)

# Usando o subconjunto de teste para validar o modelo
accuracy = classifier_model.score(features_X_test, labels_y_test)
print(accuracy)

0.9248554913294798


**Agora vamos mudar o número de vizinhos e ver como isso afeta a precisão**

In [91]:
classifier_model_3 = KNeighborsClassifier(n_neighbors=3)

classifier_model_3.fit(features_X_train, labels_y_train)
accuracy_3 = classifier_model_3.score(features_X_test, labels_y_test)
print(accuracy_3)

0.8497109826589595


In [92]:
classifier_model_7 = KNeighborsClassifier(n_neighbors=7)

classifier_model_7.fit(features_X_train, labels_y_train)
accuracy_7 = classifier_model_7.score(features_X_test, labels_y_test)
print(accuracy_7)

0.9479768786127167


In [93]:
classifier_model_9 = KNeighborsClassifier(n_neighbors=9)

classifier_model_9.fit(features_X_train, labels_y_train)
accuracy_9 = classifier_model_9.score(features_X_test, labels_y_test)
print(accuracy_9)

0.953757225433526


In [95]:
classifier_model_15 = KNeighborsClassifier(n_neighbors=15)

classifier_model_15.fit(features_X_train, labels_y_train)
accuracy_15 = classifier_model_15.score(features_X_test, labels_y_test)
print(accuracy_15)

0.8728323699421965


**Poucos vizinhos e a precisão diminui, muitos vizinhos e ainda diminui**
   
   
     Vizinhos  Precisão do modelo
        3   --->  0.84
        7   --->  0.94
        9   --->  0.95
        15  --->  0.87

### 4 - Previsões vs valores reais

In [112]:
# Fazendo isso, obteremos não apenas o número (valor_numérico_codificado), mas também o seu significado real
names = ["unacc", "acc", "good", "vgood"]

predicted_values = classifier_model.predict(features_X_test)

break_point = 0 # Não quero printar todo o DataFrame
for value in range(len(features_X_test)):
    print('Predicted value: ', predicted_values[value], '-->', names[predicted_values[value]])
    print('Input Data: ', features_X_test[value])
    print('Actual value', labels_y_test[value], '  -->   ', names[labels_y_test[value]])
    print('-'*50,'\n')
    if break_point == 4: break
    break_point += 1

Predicted value:  0 --> unacc
Input Data:  (2, 3, 2, 2, 1, 2)
Actual value 0   -->    unacc
-------------------------------------------------- 

Predicted value:  2 --> good
Input Data:  (2, 2, 1, 0, 0, 1)
Actual value 2   -->    good
-------------------------------------------------- 

Predicted value:  2 --> good
Input Data:  (0, 2, 3, 0, 2, 1)
Actual value 2   -->    good
-------------------------------------------------- 

Predicted value:  2 --> good
Input Data:  (3, 2, 2, 0, 1, 1)
Actual value 2   -->    good
-------------------------------------------------- 

Predicted value:  2 --> good
Input Data:  (3, 2, 3, 0, 0, 0)
Actual value 2   -->    good
-------------------------------------------------- 

